In [1]:
import os
from paris.models import Match
from paris.module_data import FeaturesBuilder, parse_team_name
from paris.utils import AutoBettor
import pandas as pd


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline 
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

### Features aggregation
First we get the matches from the database from CSGOLounge

In [2]:
matches = pd.DataFrame(list(Match.objects.filter(status='Results').values()))

In [3]:
matches

,team1,team2,odd1,odd2,date,logo1,logo2,status,bo,match_id,winner,time_to_go
0,nextp,xpc,2.73,1.47,2016-03-31 19:00:00+00:00,https://csgolounge.com/images/teams/go/80x80/n...,https://csgolounge.com/images/teams/go/80x80/3...,Results,BO1,2601,Team 2,
1,igg,cool,1.73,2.12,2016-04-21 17:00:00+00:00,https://csgolounge.com/images/teams/go/80x80/T...,https://csgolounge.com/images/teams/go/80x80/T...,Results,BO3,2884,Team 2,
2,agg,e-frag,3.02,1.40,2016-04-22 17:00:00+00:00,https://csgolounge.com/images/teams/go/80x80/6...,https://csgolounge.com/images/teams/go/80x80/E...,Results,BO3,2897,Team 1,
3,physx,igg,1.92,1.88,2016-04-28 17:00:00+00:00,https://csgolounge.com/images/teams/go/80x80/p...,https://csgolounge.com/images/teams/go/80x80/T...,Results,BO3,3011,Team 1,
4,igg,cool,2.10,1.74,2016-05-14 08:00:00+00:00,https://csgolounge.com/images/teams/go/80x80/T...,https://csgolounge.com/images/teams/go/80x80/T...,Results,BO1,3195,Team 2,
...,...,...,...,...,...,...,...,...,...,...,...,...
4021,winstrike,gamerlegion,1.45,2.80,2020-11-28 15:15:00+00:00,https://csgolounge.com/images/teams/go/120x120...,https://csgolounge.com/images/teams/go/120x120...,Results,BO3,259668,Team 1,
4022,alternate attax,big,7.48,1.13,2020-11-28 18:00:00+00:00,https://csgolounge.com/images/teams/go/120x120...,https://csgolounge.com/images/teams/go/120x120...,Results,BO3,259679,Team 2,
4023,1w,cyber legacy,1.49,2.67,2020-11-28 18:50:00+00:00,https://csgolounge.com/images/teams/go/120x120...,https://csgolounge.com/images/teams/go/120x120...,Results,BO3,259694,Team 2,
4024,gambit,agf,1.08,10.86,2020-11-28 18:40:00+00:00,https://csgolounge.com/images/teams/go/120x120...,https://csgolounge.com/images/teams/go/120x120...,Results,BO3,259705,Team 1,


In [9]:
FB = FeaturesBuilder()

In [10]:
FB.get_features(matches)

One or both teams not in the ranking : ['eagles', 'blink']
One or both teams not in the ranking : ['4glory', 'lvlup']
One or both teams not in the ranking : ['1w', 'cyber legacy']
One or both teams not in the ranking : ['infinity', 'supremacy (latam)']
One or both teams not in the ranking : ['furia', 'tbd']
One or both teams not in the ranking : ['mousesports', 'tbd']


,odd1,odd2,date,elo1,elo2,F_S_team1,F_S_team2
0,1.55,2.48,2020-11-28 10:00:00+01:00,1387.663830,1478.239176,2.128117,-6.136535
1,1.45,2.81,2020-11-28 13:00:00+01:00,1504.700826,1419.406427,15.855948,0.947178
3,1.20,5.15,2020-11-28 15:00:00+01:00,1713.106124,1540.225705,36.485617,43.058136
4,1.36,3.26,2020-11-28 16:00:00+01:00,1537.339451,1401.427310,7.655868,1.698500
5,2.35,1.61,2020-11-28 16:30:00+01:00,1513.208793,1726.080129,14.153647,29.132760
6,1.07,12.11,2020-11-28 16:30:00+01:00,1766.628019,1563.623890,12.333842,41.212061
8,4.56,1.23,2020-11-28 19:00:00+01:00,1506.864367,1750.824239,17.671812,24.460590
10,1.07,12.10,2020-11-28 19:30:00+01:00,1717.735778,1451.996441,12.996375,-6.593305
11,1.01,93.55,2020-11-28 20:00:00+01:00,1694.545225,1486.710858,36.261723,4.033672
12,1.63,2.29,2020-11-29 00:00:00+01:00,1402.515124,1428.768684,3.274455,16.995454


Then we compute the features with our FeaturesBuilder

In [4]:
features = FB.get_features(matches,target=True)

One or both teams not in the ranking : ['nextp', 'xpc']
One or both teams not in the ranking : ['igg', 'cool']
One or both teams not in the ranking : ['agg', 'e-frag']
One or both teams not in the ranking : ['physx', 'igg']
One or both teams not in the ranking : ['igg', 'cool']
One or both teams not in the ranking : ['nextp', 'xpc']
One or both teams not in the ranking : ['igg', 'nextp']
One or both teams not in the ranking : ['xpc', 'nextp']
One or both teams not in the ranking : ['dendd', 'agg']
One or both teams not in the ranking : ['dendd', 'agg']
One or both teams not in the ranking : ['agg', 'dendd']
One or both teams not in the ranking : ['dreams', 'mith']
One or both teams not in the ranking : ['usa fe', 'ldlc.fe']
One or both teams not in the ranking : ['ldlc.fe', 'expert.fem']
One or both teams not in the ranking : ['mith', 'dreams']
One or both teams not in the ranking : ['alternate attax', 'elements']
One or both teams not in the ranking : ['qb fire', 'elements']
One or bo

One or both teams not in the ranking : ['tyloo', 'vg']
One or both teams not in the ranking : ['evil geniuses', 'imperium (na)']


In [5]:
features

,odd1,odd2,date,elo1,elo2,F_S_team1,F_S_team2,y
8,1.36,3.27,2016-05-14 17:45:00+00:00,1421.542122,1400.000000,11.571161,0.000000,0
17,3.11,1.38,2016-12-12 21:00:00+00:00,1400.000000,1366.660453,0.000000,-5.494008,1
20,1.07,11.84,2017-01-13 13:05:00+00:00,1419.643721,1400.000000,1.410568,0.000000,1
30,2.03,1.79,2017-05-14 14:55:00+00:00,1400.000000,1350.914870,0.000000,-6.013794,1
33,1.15,6.44,2017-06-17 11:00:00+00:00,1400.000000,1400.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...
3986,2.52,1.53,2020-11-04 08:00:00+00:00,1651.615584,1749.842023,3.848309,21.573004,1
3987,1.68,2.19,2020-11-05 17:00:00+00:00,1573.648789,1400.000000,-7.365636,5.000000,0
3988,1.40,3.00,2020-11-05 17:00:00+00:00,1624.363995,1614.076948,16.191131,36.124779,1
3989,1.76,2.06,2020-11-05 20:20:00+00:00,1431.034114,1400.000000,5.811803,5.000000,1


Now we can train a simple classifier on our data

In [6]:
x_train, x_test, y_train, y_test = train_test_split(
    features.drop('y', axis=1), features['y'], test_size=0.2, shuffle=False)

In [7]:
col_trans = make_column_transformer(['drop', 'date'],
                                    remainder='passthrough')
clf = LogisticRegression()
pipeline = make_pipeline(
    (col_trans),
    (clf)
)

In [8]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop', 'drop', 'date')])),
                ('logisticregression', LogisticRegression())])

In [9]:
pipeline.score(x_test, y_test)

0.6939040207522698

Finally we can save him to use it later

In [5]:
import joblib

In [11]:
joblib.dump(pipeline, 'logit.joblib')

['logit.joblib']

In [12]:
clf2 = joblib.load('logit.joblib')